In [ ]:
import pandas as pd
import numpy as np
import tensorflow as tf


In [ ]:
from sklearn.model_selection import train_test_split
from termcolor import colored

In [ ]:
from google.colab import files
uploaded = files.upload()

Saving FinalizedFull.csv to FinalizedFull.csv


In [ ]:
df = pd.read_csv("FinalizedFull.csv", encoding = 'latin-1')

In [ ]:
print(df.head(5))

                                               tweet  senti
0  @united UA5396 can wait for me. I'm on the gro...      0
1  I hate Time Warner! Soooo wish I had Vios. Can...      0
2  @united Oh, we are sure it's not planned, but ...      0
3  Tom Shanahan's latest column on SDSU and its N...      2
4           Found the self driving car!! /IWo3QSvdu2      2


In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
from keras.preprocessing.text import Tokenizer

In [ ]:
from keras.utils import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, Embedding, LSTM, SpatialDropout1D
from sklearn.model_selection import train_test_split
import re

In [ ]:
def pre_process(tweet):
  removelist = ""
  tweet = re.sub('[^a-zA-z0-9\s]','',tweet)
  tweet = re.sub(r'@','',tweet)
  tweet = re.sub(r'#([^\s]+)', r'\1', tweet)
  tweet = re.sub(r'((www\.[^\s]+)|(https?://[^\s]+))','',tweet)
  #tweet = re.sub(r'[^w'+removelist+']', ' ',tweet)
  tweet = tweet.lower()
  return tweet
df['tweet'] = df['tweet'].apply(pre_process)
print(df.tweet)

0      united ua5396 can wait for me im on the ground...
1      i hate time warner soooo wish i had vios cant ...
2      united oh we are sure its not planned but it o...
3      tom shanahans latest column on sdsu and its nc...
4                  found the self driving car iwo3qsvdu2
                             ...                        
992    the day i never have to deal with comcast agai...
993    wish i could catch every mosquito in the world...
994    stopped to have lunch at mcdonalds chicken nug...
995    super investors a great weekend read here from...
996    boomboxhearts  im having dinner with older fri...
Name: tweet, Length: 997, dtype: object


In [ ]:
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
stop_words = set(stopwords.words('english'))
df['tweet'] = df['tweet'].apply(lambda x: ' '.join([word for word in x.split() if word not in (stop_words)]))
print(df.tweet)

0      united ua5396 wait im ground trying get gate m...
1      hate time warner soooo wish vios cant watch fr...
2      united oh sure planned occurs absolutely consi...
3      tom shanahans latest column sdsu ncaa baseball...
4                      found self driving car iwo3qsvdu2
                             ...                        
992       day never deal comcast rank one best days life
993    wish could catch every mosquito world n burn e...
994    stopped lunch mcdonalds chicken nuggetssss yum...
995    super investors great weekend read warren buff...
996    boomboxhearts im dinner older friends like 3 h...
Name: tweet, Length: 997, dtype: object


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:
nltk.download('wordnet')
w_tokenizer = nltk.tokenize.WhitespaceTokenizer()
lemmatizer = nltk.stem.WordNetLemmatizer()
def lemmatize_text(text):
    st = ""
    for w in w_tokenizer.tokenize(text):
        st = st + lemmatizer.lemmatize(w) + " "
    return st
df['tweet'] = df.tweet.apply(lemmatize_text)


[nltk_data] Downloading package wordnet to /root/nltk_data...


In [ ]:
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [ ]:

from sklearn.preprocessing import LabelEncoder
reviews = df.tweet
labels = df.senti
encoder = LabelEncoder()
encoded_labels = encoder.fit_transform(labels)
print(encoded_labels)

[0 0 0 1 1 0 0 2 2 1 1 0 1 0 1 2 0 0 2 0 2 0 2 2 1 2 1 0 1 2 2 2 1 1 1 1 1
 1 0 0 2 2 1 2 2 2 2 1 2 1 1 0 2 2 2 2 0 1 2 1 0 0 2 0 1 0 0 2 1 0 2 1 1 1
 0 0 2 0 1 2 2 1 2 2 1 0 2 0 1 0 1 2 1 1 2 2 1 2 1 2 2 0 1 1 1 0 1 0 1 1 0
 0 0 0 2 1 0 2 1 0 0 2 2 0 0 2 2 0 0 1 2 1 0 2 0 2 2 0 2 0 2 2 2 2 2 1 0 1
 1 0 1 2 2 2 0 2 0 2 0 1 2 2 0 0 0 2 0 2 2 0 0 0 2 2 1 0 2 1 2 0 2 1 2 0 1
 1 1 2 2 2 2 1 2 2 2 0 1 2 0 2 0 1 2 0 0 2 2 0 0 2 2 1 1 0 2 1 2 2 2 0 1 0
 1 0 0 2 0 1 1 0 0 2 2 0 2 0 2 2 1 2 1 0 0 0 0 0 2 1 0 0 0 1 2 2 0 2 2 2 0
 1 2 2 0 1 2 0 2 2 2 2 2 1 0 2 2 1 0 1 1 0 1 2 0 2 2 1 0 1 2 2 2 2 0 2 1 0
 2 0 1 0 2 0 1 1 0 1 2 0 0 2 0 2 2 2 0 1 2 2 0 2 2 2 1 2 1 2 0 2 1 0 2 1 2
 1 2 0 2 0 0 2 0 2 0 0 2 0 0 0 0 2 2 1 0 1 0 0 1 0 2 2 2 1 2 0 1 2 1 0 2 0
 2 2 2 1 0 0 2 0 0 2 0 2 0 1 0 0 0 1 1 0 0 1 0 0 0 0 0 2 2 0 2 2 2 1 0 0 1
 2 2 0 2 2 0 1 1 1 2 0 0 0 1 2 2 0 1 1 2 2 2 1 2 0 2 2 0 1 2 1 1 2 0 0 0 0
 0 1 0 2 2 1 2 2 2 0 2 2 1 1 0 2 1 2 0 0 2 2 2 0 1 2 2 2 0 0 0 2 1 0 2 1 1
 1 1 0 1 0 0 0 2 0 0 2 2 

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(reviews, encoded_labels, test_size = 0.20, random_state = 100)
print(X_train.shape)

(797,)


In [ ]:
vocab_size = 5000 # choose based on statistics
oov_tok = ''
embedding_dim = 100
max_length = 200 # choose based on statistics, for example 150 to 200
padding_type='post'
trunc_type='post'
# tokenize sentences
tokenizer = Tokenizer(num_words = vocab_size, oov_token=oov_tok)
tokenizer.fit_on_texts(X_train)
word_index = tokenizer.word_index
# convert train dataset to sequence and pad sequences
train_sequences = tokenizer.texts_to_sequences(X_train)
train_padded = pad_sequences(train_sequences, padding='post', maxlen=max_length)
# convert Test dataset to sequence and pad sequences
test_sequences = tokenizer.texts_to_sequences(X_test)
test_padded = pad_sequences(test_sequences, padding='post', maxlen=max_length)

In [ ]:
import keras
# model initialization
model = keras.Sequential([
    keras.layers.Embedding(vocab_size, embedding_dim, input_length=max_length),
    keras.layers.Bidirectional(keras.layers.LSTM(64)),
    keras.layers.Dense(24, activation='relu'),
    keras.layers.Dense(3, activation='sigmoid')
])
# compile model
model.compile(loss='sparse_categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])
# model summary
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 200, 100)          500000    
                                                                 
 bidirectional (Bidirection  (None, 128)               84480     
 al)                                                             
                                                                 
 dense (Dense)               (None, 24)                3096      
                                                                 
 dense_1 (Dense)             (None, 3)                 75        
                                                                 
Total params: 587651 (2.24 MB)
Trainable params: 587651 (2.24 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [ ]:
num_epochs = 10
history = model.fit(train_padded, y_train,
                    epochs=num_epochs, verbose=1,
                    validation_split=0.1)

Epoch 1/10
23/23 [==============================] - 1s 32ms/step - loss: 0.0057 - accuracy: 0.9986 - val_loss: 1.2808 - val_accuracy: 0.6750
Epoch 2/10
23/23 [==============================] - 1s 41ms/step - loss: 0.0036 - accuracy: 1.0000 - val_loss: 1.3192 - val_accuracy: 0.6875
Epoch 3/10
23/23 [==============================] - 1s 41ms/step - loss: 0.0026 - accuracy: 1.0000 - val_loss: 1.3631 - val_accuracy: 0.6875
Epoch 4/10
23/23 [==============================] - 1s 25ms/step - loss: 0.0023 - accuracy: 1.0000 - val_loss: 1.4029 - val_accuracy: 0.6875
Epoch 5/10
23/23 [==============================] - 1s 40ms/step - loss: 0.0017 - accuracy: 1.0000 - val_loss: 1.4387 - val_accuracy: 0.6750
Epoch 6/10
23/23 [==============================] - 1s 32ms/step - loss: 0.0012 - accuracy: 1.0000 - val_loss: 1.4697 - val_accuracy: 0.6875
Epoch 7/10
23/23 [==============================] - 1s 24ms/step - loss: 9.7320e-04 - accuracy: 1.0000 - val_loss: 1.4992 - val_accuracy: 0.6750
Epoch 8/1

In [ ]:
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score

In [ ]:
result = model.evaluate(test_padded, y_test)

7/7 [==============================] - 0s 13ms/step - loss: 1.1748 - accuracy: 0.7200


In [ ]:
y_pred = model.predict(test_padded)
print(X_train , y_train)
print(y_pred)

7/7 [==============================] - 0s 12ms/step
159    click link want flight info havent booked flig...
500                         need selfdriving car google 
402    goodby silverstein agency new site httpwwwgood...
551    republican party bunch antiabortion zealot cou...
25     omg would died actually take backi keep update...
                             ...                        
855               jdreiss oh yes gm dy worth boo hahaha 
871    north korea please cease douchebaggery china d...
835    rt justindavey rt tweetmeme gm onstar instantl...
792                     finally work back fucking wreck 
520         last day work go holiday argh got fuck load 
Name: tweet, Length: 797, dtype: object [1 2 2 0 2 0 2 0 1 1 2 0 2 1 1 0 0 0 2 2 1 0 1 0 2 2 2 0 2 1 2 0 1 2 1 0 0
 1 2 0 0 1 0 1 0 2 0 1 0 2 1 0 1 0 2 2 0 2 1 1 0 1 2 2 0 0 2 2 2 0 1 1 2 2
 2 2 0 2 1 2 2 2 2 0 2 2 0 1 1 1 0 0 0 2 2 2 0 0 1 1 1 2 1 2 0 1 0 2 2 1 0
 0 2 2 2 0 0 0 1 0 1 2 0 2 0 0 1 2 0 1 2 2 0 0 1 0 2 0 2

In [ ]:
!pip install -U scikit-fuzzy
import skfuzzy as fuzz

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 994.0/994.0 kB 4.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for scikit-fuzzy: filename=scikit_fuzzy-0.4.2-py3-none-any.whl size=894078 sha256=9771ad54d236bc93cd675b035c99b0a19a8f782344393270be1df45da1478f00
  Stored in directory: /root/.cache/pip/wheels/4f/86/1b/dfd97134a2c8313e519bcebd95d3fedc7be7944db022094bc8
Successfully built scikit-fuzzy


In [ ]:
input1_positive = np.arange(0,1,0.1)
input2_negative = np.arange(0,1,0.1)
sentiment_output = np.arange(0,10,1)

In [ ]:
#triangular membership function for positive input
pos_low = fuzz.trapmf(input1_positive, [0,0,0.4,0.6])
#trapezoid membership function for medium positive input
pos_med = fuzz.trapmf(input1_positive, [0,0.4,0.5,1])
#trapezoid membership function for high positive input
pos_high = fuzz.trapmf(input1_positive, [0.5,0.55,1,1])
#triangular membership function for low negative input
neg_low = fuzz.trapmf(input2_negative, [0,0,0.2,0.3])
#trapezoid membership function for medium negative input
neg_med = fuzz.trapmf(input2_negative, [0,0.3,0.5,1])
#trapezoid membership function for high negative input
neg_high = fuzz.trapmf(input2_negative, [0.5,0.55,1,1])


op_neg = fuzz.trapmf(sentiment_output, [0,0,0,5])
op_neu = fuzz.trapmf(sentiment_output, [0,5,5,10])
op_pos = fuzz.trapmf(sentiment_output, [5,5,5,10])

In [ ]:
sentiment = []
for i in range(len(test_padded)):
  pos_score = y_pred[i][2]

  neg_score = y_pred[i][0]
  if pos_score > neg_score:
    neg_score = 1 - pos_score
  else:
    pos_score = 1 - neg_score

  p_level_lo = fuzz.interp_membership(input1_positive, positive_low, pos_score)
  p_level_md = fuzz.interp_membership(input1_positive, positive_medium, pos_score)
  p_level_hi = fuzz.interp_membership(input1_positive, positive_high, pos_score)

  n_level_lo = fuzz.interp_membership(input2_negative, negative_low, neg_score)
  n_level_md = fuzz.interp_membership(input2_negative, negative_medium, neg_score)
  n_level_hi = fuzz.interp_membership(input2_negative, negative_high, neg_score)
  active_rule1 = np.fmin(p_level_lo, n_level_lo)
  active_rule2 = np.fmin(p_level_md, n_level_lo)
  active_rule3 = np.fmin(p_level_hi, n_level_lo)
  active_rule4 = np.fmin(p_level_lo, n_level_md)
  active_rule5 = np.fmin(p_level_md, n_level_md)
  active_rule6 = np.fmin(p_level_hi, n_level_md)
  active_rule7 = np.fmin(p_level_lo, n_level_hi)
  active_rule8 = np.fmin(p_level_md, n_level_hi)
  active_rule9 = np.fmin(p_level_hi, n_level_hi)

  n1=np.fmax(active_rule4,active_rule7)
  n2=np.fmax(n1,active_rule8)
  op_activation_lo = np.fmin(n2,negative_output)

  neu1=np.fmax(active_rule1,active_rule5)
  neu2=np.fmax(neu1,active_rule9)
  op_activation_md = np.fmin(neu2,neutral_output)

  p1=np.fmax(active_rule2,active_rule3)
  p2=np.fmax(p1,active_rule6)
  op_activation_hi = np.fmin(p2,positive_output)

  op0 = np.zeros_like(sentiment_output)

    # Aggregate all three output membership functions together
  aggregated = np.fmax(op_activation_lo,
                         np.fmax(op_activation_md, op_activation_hi))

    # Calculate defuzzified result
  op = fuzz.defuzz(sentiment_output, aggregated, 'lom')
  if 0<=(op)<=3.33:    # R
        #print("\nOutput after Defuzzification: Negative")
        sentiment.append(0)

  elif 3.34<=(op)<=6.66:
        #print("\nOutput after Defuzzification: Neutral")
        sentiment.append(1)

  elif 6.67<=(op)<=10:
        #print("\nOutput after Defuzzification: Positive")
        sentiment.append(2)


In [ ]:
from sklearn.metrics import f1_score, precision_score, recall_score, accuracy_score

y_true = y_test
y_pred = sentiment
print(y_test)
print(y_pred)
acc=accuracy_score(y_true, y_pred)
print("Accuracy score:",str(round(acc*100,3)))

[1 2 2 0 0 0 2 2 1 2 2 0 2 1 2 0 0 0 2 2 1 1 2 2 0 2 0 2 0 2 1 1 1 0 0 0 1
 2 0 2 0 0 0 0 2 1 0 0 2 2 0 0 0 0 2 2 2 0 2 0 0 2 1 1 2 1 1 0 1 2 0 1 0 2
 2 2 0 0 0 1 0 2 2 2 1 0 2 0 0 0 1 2 0 2 0 2 0 2 1 2 0 0 0 2 0 0 0 0 0 2 1
 0 2 0 0 2 1 2 1 0 0 2 2 0 0 2 2 0 2 2 0 1 0 1 0 0 2 0 0 0 0 2 2 2 2 1 2 2
 1 1 0 1 2 0 1 0 2 2 0 2 2 1 2 1 0 1 2 0 0 0 1 2 0 1 0 2 2 2 1 1 0 0 1 1 0
 1 2 0 1 0 0 2 0 0 0 0 1 2 1 1]
[2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 1, 2, 2, 0, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 0, 2, 0, 2, 2, 2, 2, 2, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 1, 2, 1, 2, 2, 2, 0, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 0, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 1, 2, 2, 0, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 0, 2, 2, 0, 2, 2, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2